For a full description and infographic of the shunPykeR pipeline, please check the Github repository main page (https://github.com/kousaa/shunPykeR).

<a id='contents'></a>

##### Notebook setup

Load python packages and modules that are used across this notebook. For an introduction to python packages and modules, please see https://realpython.com/python-modules-packages/#python-packages.    

In [ ]:
import scanpy as sc
import scanpy.external as sce
#import scvelo as scv
import numpy as np
import pandas as pd
import warnings, scipy.sparse as sp, matplotlib, matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.pyplot import rc_context
from collections import Counter
from pathlib import Path
from sklearn.metrics.pairwise import cosine_distances
#from mousipy import translate

import matplotlib.font_manager
#import openpyxl
#import pyreadr
#import rpy2
#from rpy2.robjects.packages import importr
#import rpy2.robjects as robjects
#import seaborn as sns
#import palantir
#import loompy
#import feather
import re
#from scipy.sparse import csgraph

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rc('font', size=14)
import matplotlib.lines as lines

sc.set_figure_params(dpi=80, dpi_save=300, color_map='Spectral_r', vector_friendly=True, transparent=True)
sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

#scv.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True, format='pdf')

pd.set_option('display.max_rows', 200)

In [ ]:
def observe_variance(anndata_object):
    fig = plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    # variance per principal component
    x = range(len(anndata_object.uns['pca']['variance_ratio']))
    y = anndata_object.uns['pca']['variance_ratio']
    ax1.scatter(x,y,s=4)
    ax1.set_xlabel('PC')
    ax1.set_ylabel('Fraction of variance explained\n')
    ax1.set_title('Fraction of variance explained per PC\n')
    # cumulative variance explained
    cml_var_explained = np.cumsum(anndata_object.uns['pca']['variance_ratio'])
    x = range(len(anndata_object.uns['pca']['variance_ratio']))
    y = cml_var_explained
    ax2.scatter(x,y,s=4)
    ax2.set_xlabel('PC')
    ax2.set_ylabel('Cumulative fraction of variance\nexplained')
    ax2.set_title('Cumulative fraction of variance\nexplained by PCs')
    fig.tight_layout()
    plot = plt.show
    return(plot)

In [ ]:

#!{sys.executable} -m pip install xlsxwriter
#!{sys.executable} -m pip install rpy2
import xlsxwriter
import rpy2

In [ ]:
import os

os.environ['R_HOME'] = '/Library/Frameworks/R.framework/Resources'

In [ ]:
%load_ext rpy2.ipython

%R if (!require("pacman")) install.packages("pacman")
%R pacman::p_load(MAST, scales, data.table, openxlsx, ggplot2, ggpubr, RColorBrewer, dichromat, readxl, ggpubr, pheatmap, dplyr, arrow, feather, DelayedArray, HDF5Array, stringr, parallel)

`%matplotlib inline` is a magic command that renders the figures within the notebook. For more information regarding python's magic operators, see https://ipython.readthedocs.io/en/stable/interactive/magics.html.

In [ ]:
%matplotlib inline 

To get you started in a visually pleasing way, here are some manually curated color palettes. Feel free to use the ones provided here, the python predefined ones here (https://matplotlib.org/stable/tutorials/colors/colormaps.html) or make up your own.

In [ ]:
# divergent
user_defined_palette =  [ '#3283FE', '#16FF32', '#F6222E',  '#FEAF16', '#BDCDFF', '#3B00FB', '#1CFFCE', '#C075A6', '#F8A19F', '#B5EFB5', '#FBE426', '#C4451C', '#2ED9FF', '#c1c119', '#8b0000', '#FE00FA', '#1CBE4F', '#1C8356', '#0e452b', '#AA0DFE', '#B5EFB5', '#325A9B', '#90AD1C']
# gradient of one color
user_defined_cmap_markers = LinearSegmentedColormap.from_list('mycmap', ["#E6E6FF", "#CCCCFF", "#B2B2FF", "#9999FF",  "#6666FF",   "#3333FF", "#0000FF"])
# gradient of two colors
user_defined_cmap_degs = LinearSegmentedColormap.from_list('mycmap', ["#0000FF", "#3333FF", "#6666FF", "#9999FF", "#B2B2FF", "#CCCCFF", "#E6E6FF", "#E6FFE6", "#CCFFCC", "#B2FFB2", "#99FF99", "#66FF66", "#33FF33", "#00FF00"])

💡 **Hint:** Use the code below ONLY if you need to install extra packages

In [ ]:
# import sys
# !{sys.executable} -m pip install scrublet

Return to the beginning of this notebook <a href='#contents'>here</a>.

<a id='QC'></a>

## Perform quality control and clean-up samples

### Load cellranger files

Start by converting any cellranger's (https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/output/overview) Hierarchical Data Format 5 (H5 or h5) files into a scRNA-seq friendly python object called an anndata object. This object can store different types of information under its multiple compartments. For a more detailed explanation, see https://anndata.readthedocs.io/en/latest/anndata.AnnData.html#anndata.AnnData. 

The example below uses Peripheral Blood Mononuclear Cells (PBMCs) datasets from the 10x Genomics public repository, processed with cellranger v2 [⬇️](https://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_1k_v2/pbmc_1k_v2_filtered_feature_bc_matrix.h5) and v3 [⬇️](https://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_1k_v3/pbmc_1k_v3_filtered_feature_bc_matrix.h5) to perform a mock analysis).

✍️ Provide the path to your cellranger's `filtered_feature_bc.h5` files in the `tenexdir` row below. The code will search that directory and store all h5 files in a python [list](https://realpython.com/python-lists-tuples/), names here `adatas_list` as it contains multiple adata (short for anndata) objects.

In [ ]:
# adatas_list=[]

# tenexdir = '/Users/brandonng/Google Drive/Weill Cornell/LAB/folder containing all data from Mac pro/experiments and data/single cell/scotts single cell data/Project_15143_B/'


APRIL_28_1 = sc.read_10x_h5('/Users/brandonng/Google Drive/Weill Cornell/LAB/folder containing all data from Mac pro/experiments and data/single cell/20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/seq_files/Project_15459_B/pipeline/CI82_spl1/per_sample_outs/APRIL_28_rep_1/sample_filtered_feature_bc_matrix.h5')
APRIL_28_2 = sc.read_10x_h5('/Users/brandonng/Google Drive/Weill Cornell/LAB/folder containing all data from Mac pro/experiments and data/single cell/20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/seq_files/Project_15459_B/pipeline/CI82_spl2/per_sample_outs/APRIL_28_rep2/sample_filtered_feature_bc_matrix.h5')

APRIL_28_IL18_1 = sc.read_10x_h5('/Users/brandonng/Google Drive/Weill Cornell/LAB/folder containing all data from Mac pro/experiments and data/single cell/20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/seq_files/Project_15459_B/pipeline/CI82_spl1/per_sample_outs/APRIL_28_18_rep_1/sample_filtered_feature_bc_matrix.h5')
APRIL_28_IL18_2 = sc.read_10x_h5('/Users/brandonng/Google Drive/Weill Cornell/LAB/folder containing all data from Mac pro/experiments and data/single cell/20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/seq_files/Project_15459_B/pipeline/CI82_spl2/per_sample_outs/APRIL_28_18_rep2/sample_filtered_feature_bc_matrix.h5')

APRIL_BB_1 = sc.read_10x_h5('/Users/brandonng/Google Drive/Weill Cornell/LAB/folder containing all data from Mac pro/experiments and data/single cell/20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/seq_files/Project_15459_B/pipeline/CI82_spl1/per_sample_outs/APRIL_BB_rep_1/sample_filtered_feature_bc_matrix.h5')
APRIL_BB_2 = sc.read_10x_h5('/Users/brandonng/Google Drive/Weill Cornell/LAB/folder containing all data from Mac pro/experiments and data/single cell/20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/seq_files/Project_15459_B/pipeline/CI82_spl2/per_sample_outs/APRIL_BB_rep2/sample_filtered_feature_bc_matrix.h5')

APRIL_BB_IL18_1 = sc.read_10x_h5('/Users/brandonng/Google Drive/Weill Cornell/LAB/folder containing all data from Mac pro/experiments and data/single cell/20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/seq_files/Project_15459_B/pipeline/CI82_spl1/per_sample_outs/APRIL_BB_18_rep_1/sample_filtered_feature_bc_matrix.h5')
APRIL_BB_IL18_2 = sc.read_10x_h5('/Users/brandonng/Google Drive/Weill Cornell/LAB/folder containing all data from Mac pro/experiments and data/single cell/20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/seq_files/Project_15459_B/pipeline/CI82_spl2/per_sample_outs/APRIL_BB_18_rep2/sample_filtered_feature_bc_matrix.h5')



APRIL_28_1.var_names_make_unique()
APRIL_28_2.var_names_make_unique()
APRIL_28_IL18_1.var_names_make_unique()
APRIL_28_IL18_2.var_names_make_unique()
APRIL_BB_1.var_names_make_unique()
APRIL_BB_2.var_names_make_unique()
APRIL_BB_IL18_1.var_names_make_unique()
APRIL_BB_IL18_2.var_names_make_unique()


# a for loop to read and store all .h5 files under the tenexdir path
# for path in h5_path:
#     tmp_adata = sc.read_10x_h5(path)
#     tmp_adata.var_names_make_unique()
#     print(tmp_adata.shape) # check the number of cells and genes in each sample
#     adatas_list.append(tmp_adata)

In [ ]:

APRIL_28_1.obs['sample']='APRIL_28'
APRIL_28_2.obs['sample']='APRIL_28'
APRIL_28_IL18_1.obs['sample']='APRIL_28_IL18'
APRIL_28_IL18_2.obs['sample']='APRIL_28_IL18'
APRIL_BB_1.obs['sample']='APRIL_BB'
APRIL_BB_2.obs['sample']='APRIL_BB'
APRIL_BB_IL18_1.obs['replicate']='APRIL_BB_IL18'
APRIL_BB_IL18_2.obs['replicate']='APRIL_BB_IL18'

APRIL_28_1.obs['replicate']='APRIL_28_1'
APRIL_28_2.obs['replicate']='APRIL_28_2'
APRIL_28_IL18_1.obs['replicate']='APRIL_28_IL18_1'
APRIL_28_IL18_2.obs['replicate']='APRIL_28_IL18_2'
APRIL_BB_1.obs['replicate']='APRIL_BB_1'
APRIL_BB_2.obs['replicate']='APRIL_BB_2'
APRIL_BB_IL18_1.obs['replicate']='APRIL_BB_1'
APRIL_BB_IL18_2.obs['replicate']='APRIL_BB_IL18_2'


✍️ The next command will take as input the various anndata objects that have been created above, concatenate them and create a metadata field using the `label` field. Use then the `keys` field to annotate each sample accordingly. The order of the samples is defined by the order that the h5 files are stored in the function above.

In [ ]:
APRIL_28_concat = sc.concat(
    (APRIL_28_1, APRIL_28_2),
    join='outer', 
    label = 'replicate', 
    keys = ['APRIL_28_1', 'APRIL_28_2'],
    index_unique = '@'
)

In [ ]:
APRIL_28_concat.obs

In [ ]:
APRIL_28_IL18_concat = sc.concat(
    (APRIL_28_IL18_1, APRIL_28_IL18_2),
    join='outer', 
    label = 'replicate', 
    keys = ['APRIL_28_IL18_1', 'APRIL_28_IL18_2'],
    index_unique = '@'
)

In [ ]:
APRIL_28_IL18_concat.obs

In [ ]:
APRIL_BB_concat = sc.concat(
    (APRIL_BB_1, APRIL_BB_2),
    join='outer', 
    label = 'replicate', 
    keys = ['APRIL_BB_1', 'APRIL_BB_2'],
    index_unique = '@'
)

In [ ]:
APRIL_BB_concat.obs

In [ ]:
APRIL_BB_IL18_concat = sc.concat(
    (APRIL_BB_IL18_1, APRIL_BB_IL18_2),
    join='outer', 
    label = 'replicate', 
    keys = ['APRIL_BB_IL18_1', 'APRIL_BB_IL18_2'],
    index_unique = '@'
)

In [ ]:
APRIL_BB_IL18_concat.obs

In [ ]:
adata = sc.concat(
    (APRIL_28_concat, APRIL_28_IL18_concat, APRIL_BB_concat, APRIL_BB_IL18_concat),
    join='outer', 
    label = 'sample', 
    keys = ['APRIL_28', 'APRIL_28_IL18', 'APRIL_BB', 'APRIL_BB_IL18'],
    index_unique = '@'
)

In [ ]:
adata.obs

In [ ]:
adata.raw = adata # keep a copy of the raw adata 
np.random.seed(42) 
index_list = np.arange(adata.shape[0]) # randomize the order of cells for plotting
np.random.shuffle(index_list)
adata = adata[index_list]

Once you execute either of the input commands above, you should be able to access:
- gene names: type `adata.var` + click `Run`
- cell index and other annotation: type `adata.obs` + click `Run`
- total number of cells (x, ) and genes ( ,y): type `adata.shape` + click `Run`

❗ The latter one is a good sanity check especially after filtering steps to make sure the filters have been applied properly. 

In [ ]:
adata.shape

### Calculate quality control metrics and perform standard clean-up

We will now calculate some standard quality control (QC) metrics using the `calculate_qc_metrics` function that is provided by scanpy. These metrics include total counts, total number of genes, ribosomal, mitochondrial and hemoglobin fractions on a per cell basis. Summarized violin plots of these metrics can be also visualized below on a per sample basis.

In [ ]:
sc.pp.calculate_qc_metrics(adata, inplace=True)

#store unfiltered/unprocessed data into new fields prior to downstream analysis
adata.obs['original_total_counts'] = adata.obs['total_counts']
adata.obs['log10_original_total_counts'] = np.log10(adata.obs['original_total_counts'])

# select mitochondrial, ribosomal and hemoglobin genes (case sensitive)
adata.var['mt'] = adata.var_names.str.startswith(('MT-', 'mt-'))
adata.var['ribo'] = adata.var_names.str.startswith(('RPS','RPL', 'Rps', 'Rpl'))
adata.var['hb'] = adata.var_names.str.startswith(('HB', 'Hb'))

# for each cell compute fraction of counts in mitochondrial, ribosomal and hemoglobin genes vs. all genes 
adata.obs['mito_frac'] = np.sum(adata[:,adata.var['mt']==True].X, axis=1) / np.sum(adata.X, axis=1)
adata.obs['ribo_frac'] = np.sum(adata[:,adata.var['ribo']==True].X, axis=1) / np.sum(adata.X, axis=1)
adata.obs['hb_frac'] = np.sum(adata[:,adata.var['hb']==True].X, axis=1) / np.sum(adata.X, axis=1)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'log10_original_total_counts', 'ribo_frac', 'mito_frac', 'hb_frac'],  
             palette=user_defined_palette,  jitter=0.4, groupby = 'replicate', rotation= 90)

#### Identify doublets

We will use scanpy's implementation (`scanpy.external.pp.scrublet` function) of scrublet (https://github.com/swolock/scrublet) to predict doublets. This command will use the default parameters of `scanpy.external.pp.scrublet`, however, if you are feeling brave and disagree with the predicted parameters you can use this scrublet's example notebook (https://github.com/swolock/scrublet/blob/master/examples/scrublet_basics.ipynb) to alter them to your preferred ones.

In [ ]:
sc.external.pp.scrublet(adata, random_state=42) # run scrublet to predict potential doublets

In [ ]:
sc.external.pl.scrublet_score_distribution(adata) # inspect automatic threshold

✍️ Because automatic threshold was not optimal, here we adjusted the threshold manually to make it more stringent.

In [ ]:
sc.external.pp.scrublet(adata, threshold=0.25, random_state=42) # choose threshold manually

In [ ]:
sc.external.pl.scrublet_score_distribution(adata) # inspect manual threshold

#### Remove not expressed genes

✍️ We will remove genes that are not expressed in any cells across our samples, as these genes will not contribute to any biological insight, but rather may affect downstream analysis speed. This step can become more stringent by providing a higher number of cells in the `min_cells` field.

In [ ]:
sc.pp.filter_genes(adata, min_cells=1) # remove columns with all 0s

#### Remove ribosomal and hemoglobin genes

Ribosomal and hemoglobin genes constitute a high percentage of the total read counts, and they can affect downstream analysis and interpretation of our data, therefore, here we remove these genes before further processing.

In [ ]:
adata = adata[:,adata.var['ribo']==False]
adata.shape # check number of genes after ribosomal gene removal

In [ ]:
adata = adata[:,adata.var['hb']==False]
adata.shape # check number of genes after hemoglobin gene removal

#### Normalize each cell's library size

Next, we will normalize each cell to a total library size of 10,000 reads. The `adata.X` field will be updated automatically with the normalized counts.

In [ ]:
sc.pp.normalize_per_cell(adata, counts_per_cell_after=10**4)

❗ Raw data can be recovered via the `adata.raw` field.

#### Log-transform counts

We will also *log*-transform the gene counts here using scanpy's `sc.pp.log1p` function. This function uses the `log(X+1)` formula, in which *log* denotes the natural logarithm.

In [ ]:
sc.pp.log1p(adata)

In [ ]:
adata.obs

### Select a subset of principal components 

Prior to plotting our data in a 2-dimensional space (in this case using scanpy's `sc.pl.umap` function), we need to select a small number of principal components to reduce the noise of this multitude of data and as well increase the execution speed.

For that, we will first select a large number of principal components and visualize the percentage of variance explained by each component.

*For fun, here and in all the cases that we need to set a seed, we will use the **number 42**, the answer to the Ultimate Question of Life, the Universe, and Everything.*

In [ ]:
rng = np.random.RandomState(42) # set seed 
sc.tl.pca(adata, n_comps=200, svd_solver='arpack', random_state=rng)

In [ ]:
observe_variance(adata)

❗ We want to include all the principal components before the 'knee point' (the point where the fitted curve would change slope the most), because these likely represent important sources of variance in our data. To try and avoid eliminating any relevant but small sources of variance, we will choose a number of PCs slightly to the right of the knee point. 

✍️ Replace the number in the `n_comps` field with your principal components selection.

In [ ]:
rng = np.random.RandomState(42) # set seed 
sc.tl.pca(adata, n_comps=50,  svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=15, random_state=42) # set seed
sc.tl.umap(adata)

In [ ]:
observe_variance(adata)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata,
    color=['sample'], 
    color_map='Spectral_r', 
    palette=user_defined_palette,
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=30,
    frameon=False,
    add_outline=True,
    sort_order = False
)

❗ **Does your data look well distinguished?** If yes, move ahead. If no, maybe try to increase (or decrease) the number of principal components you used, and rerun the last bits of code.

### Run unsupervised clustering analysis with leiden

Next, we perform leiden clustering analysis using consecutive resolution parameters and visualize the resulting clusters in the respective generated UMAPs. We can then visually select an optimal number of clusters that better identify contaminant populations and also clusters of cells that have a combination of "bad" quality measures. Here, we aim to remove as cleanly as possible bad quality cells and cell contaminants that we don't want carry in our downstream analysis.   

In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(adata, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

We can visualize the clustering to see which clusters match with the cells that we would like to filter out. Inspect the list of QC metrics and canonical markers to make your choice.

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8', 'leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.3,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['leiden_0.2'], 
    palette=user_defined_palette,  
    color_map='Spectral_r',
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['Cd3e','Cd4', 'Cd8a', 'Sdc1'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.2,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False,
    vmax=2
)

❗ Store total number of cells prior to any filtering to use for a per sample summary cells that did not pass QC visual

In [ ]:
adata_memo = adata

### Filter out bad quality cells and other cell contaminants by cluster

Now inspect the QC metrics for each of the clusters and choose to remove any clusters that may have a combination of bad quality metrics.

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['log10_original_total_counts', 'n_genes_by_counts','ribo_frac', 'mito_frac'], 
    palette=user_defined_palette,  
    color_map='Spectral_r',
    use_raw=False,
    ncols=5,
    wspace = 0.2,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.tl.rank_genes_groups(mystery_cells, 'sample', method='wilcoxon', use_raw=False)
result = mystery_cells.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names']}).head(100)

✍️ Choose which clusters you want to remove, and add them to the list clusters_to_remove below.

In [ ]:
clusters_to_remove = ['3']
cluster_filter = [x not in clusters_to_remove for x in adata.obs['leiden_0.2']]
print('Total number of cells pre-filtering: ' + str(adata.shape[0]))
print('Number of cells to keep after filtering: ' + str(sum(cluster_filter)))
adata_filtered = adata[cluster_filter]

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata_filtered, 
    color=['leiden_0.2'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

❗ **Were the correct clusters removed?** If the bad quality cells and doublets are removed, run the next line to store the result in your adata object and continue with the analysis.

In [ ]:
adata = adata_filtered

In [ ]:
adata.shape

### Filter out doublets

This would be also a good spot to remove the doublets that we have identified above. We first inspect where the predicted doublets lie on the UMAP and then we proceed to remove them.

In [ ]:
sc.pl.umap(
    adata, 
    color=['predicted_doublet', 'doublet_score'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
adata = adata[adata.obs['predicted_doublet'] == False] # remove cells annotated as doublets

In [ ]:
sc.pl.umap(
    adata, 
    color=['predicted_doublet', 'doublet_score'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

Let's now check the exact number of doublet cells that were removed from the dataset.

In [ ]:
adata.shape

### Reanalyze the data after removal of "unwanted" cells

Once we have removed all "unwanted" cells, we need to reanalyze our data in a similar way to sections 1.3 and 1.4 above. 

In [ ]:
rng = np.random.RandomState(42)
adata = sc.tl.pca(adata, n_comps=200, copy = True, svd_solver='arpack', random_state=rng)

In [ ]:
observe_variance(adata)

✍️ Remember to choose here a different number of principal components based on the re-analysis.

In [ ]:
rng = np.random.RandomState(42) # use seed
sc.tl.pca(adata, n_comps=55, svd_solver='arpack', random_state=rng)
sc.pp.neighbors(adata, n_neighbors=15)
sc.tl.umap(adata)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['sample'], 
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=25,
    frameon=False,
    add_outline=False,
    sort_order = False
)

As previously, we perform leiden clustering analysis using consecutive resolution parameters and visualize the resulting clusters in the respective generated UMAPs. Here, we opt to select the number of clusters that better previously known populations (based on the expression of canonical markers), and also *de-novo* populations of cells that look well separated in the UMAP representation.

In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(adata, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8', 'leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    ncols=5,
    wspace = 0.3,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['Cd3e', 'Cd4', 'Cd8a', 'Sdc1'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False,
    vmax=2
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['sample'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False,
    vmax=2
)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'log10_original_total_counts', 'ribo_frac', 'mito_frac', 'hb_frac'],  
             palette=user_defined_palette,  jitter=0.4, groupby = 'leiden_0.2', rotation= 90)

In [ ]:
clusters_to_remove = ['4']
cluster_filter = [x not in clusters_to_remove for x in adata.obs['leiden_0.2']]
print('Total number of cells pre-filtering: ' + str(adata.shape[0]))
print('Number of cells to keep after filtering: ' + str(sum(cluster_filter)))
adata_filtered = adata[cluster_filter]

In [ ]:
adata = adata_filtered

✍️ Remember to choose here a different number of principal components based on the re-analysis.

In [ ]:
rng = np.random.RandomState(42) # use seed
sc.tl.pca(adata, n_comps=55, svd_solver='arpack', random_state=rng)
# sc.pp.neighbors(adata, n_neighbors=100)
# sc.tl.umap(adata, min_dist=0.5)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=38)

In [ ]:
sc.tl.umap(adata, min_dist=0.2)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['sample', 'Cd4', 'Cd8a', 'Itga2'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.4,
    outline_width=[0.6, 0.05],
    size=20,
    frameon=False,
    add_outline=False,
    sort_order = False,
)

In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(adata, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8', 'leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    ncols=5,
    wspace = 0.3,
    outline_width=[0.6, 0.05],
    size=7,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
clusters_to_remove = ['2']
cluster_filter = [x not in clusters_to_remove for x in adata.obs['leiden_0.1']]
print('Total number of cells pre-filtering: ' + str(adata.shape[0]))
print('Number of cells to keep after filtering: ' + str(sum(cluster_filter)))
adata_filtered = adata[cluster_filter]

In [ ]:
adata = adata_filtered

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['n_genes_by_counts', 'log10_original_total_counts', 'ribo_frac', 'mito_frac', 'hb_frac'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False,
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['Cd3e', 'Cd4', 'Cd8a', 'Sdc1'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False,
    vmax=2
)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=55)

In [ ]:
sc.tl.umap(adata, min_dist=0.2, spread = 5)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['sample'], 
    use_raw=False,
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    ncols=5,
    wspace = 0.4,
    outline_width=[0.6, 0.05],
    size=30,
    frameon=False,
    add_outline=False,
    sort_order = False,
)

### Inspect the data after removal of "unwanted" cells

Here, we create annotation for the cells that have been removed or kept to visualize them in a UMAP and a stacked barplot format.

In [ ]:
adata.obs['keep_cell'] = '0.yes'
keep_cells = pd.concat([adata.obs['keep_cell']])
adata_memo.obs['keep_cell'] = '1.no'

In [ ]:
adata_memo.obs['keep_cell'][adata_memo.obs.index.isin(keep_cells.index) == True] = '0.yes'

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(
    adata_memo, 
    color=['keep_cell'], 
    palette=['blue',  '#d3d3d3'],  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
crosstb = pd.crosstab(adata_memo.obs['sample'], adata_memo.obs['keep_cell'])

In [ ]:
with rc_context({'figure.figsize': (2, 3)}):
    ax = crosstb.plot(kind="bar", stacked=True, edgecolor = "black", width=0.8,  color=['blue', '#d3d3d3'])
    ax.grid(False) 
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    # to save plot unhash the line bellow
#     plt.savefig('202312_CI82_barplot_keepcells.pdf', bbox_inches = 'tight')

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    adata, 
    color=['n_genes_by_counts', 'log10_original_total_counts', 'ribo_frac', 'mito_frac', 'hb_frac'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.1,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=False,
    sort_order = False,
#     save='_CI82_QC_metrics.pdf'
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.violin(adata, ['n_genes_by_counts', 'log10_original_total_counts', 'ribo_frac', 'mito_frac', 'hb_frac'],  
             palette=user_defined_palette,  jitter=0.4, groupby = 'sample', rotation= 90, wspace = 0.5,
#             save="CI82_QC_metrics.pdf"
            )

You are done! Just save you anndata object to come back to it later. Make sure your path ends with the desired filename including the .h5ad file extension. You can now come back to this dataset at anytime.

Return to the beginning of this notebook <a href='#contents'>here</a>.

<a id='phase'></a>

Return to the beginning of this notebook <a href='#contents'>here</a>.

<a id='harmony'></a>

## perform clustering using highly variable genes

When ispecting the annotated by sample UMAP above, it is likely that samples from different experiments do not properly overlap, despite having shared populations, due to small but consistent technical effects (batches) across each dataset that influence their projection. We can initially attempt to correct for this by choosing only highly variable genes (hvgs) when running scanpy's `sc.tl.pca` function. This will create the UMAP based on the biggest differences of the existing populations and may salvage the technical noise that is more apparent when running the  analysis across all genes.

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000, n_bins=20, flavor='seurat',  inplace=True)

As above, observe and choose an optimal number of components based on the "knee point" of the curve below.

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(adata, n_comps=200, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
observe_variance(adata)

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(adata, n_comps=75, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
sc.tl.umap(adata, min_dist=0.2, spread = 5)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)
sc.pl.umap(
    adata, 
    color=['sample'], 
    use_raw=False,
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    ncols=5,
    wspace = 0.4,
    outline_width=[0.6, 0.05],
    size=30,
    frameon=False,
    add_outline=False,
    sort_order = False,
#     save='_by_sample_2.pdf'
)

### Rerun leiden clustering

Once we have updated the pca components we use to compute the UMAP, we need to rerun leiden clustering analysis using consecutive resolution parameters and visualize the resulting clusters in the respective generated UMAPs to choose an appropriate UMAP.

In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(adata, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8', 'leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r',
    use_raw=False,
    ncols=5,
    wspace = 0.3,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    adata, 
    color=['leiden_1.0', 'sample'], 
    palette=user_defined_palette,  
    color_map='Spectral_r',
    use_raw=False,
    ncols=5,
    wspace = 0.3,
    outline_width=[0.6, 0.05],
    size=40,
    frameon=False,
    add_outline=False,
    sort_order = False
)

In [ ]:
path_to_h5ad = '/Users/brandonng/Google Drive/Weill Cornell/LAB/folder containing all data from Mac pro/experiments and data/single cell/20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal.h5ad'


In [ ]:
# adata.write(path_to_h5ad)

In [ ]:
# code to read back your saved h5ad object
adata = sc.read_h5ad(path_to_h5ad)
adata.uns['log1p']["base"] = None

In [ ]:
adata.shape

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.umap(adata[adata.obs['sample']=='APRIL_28'],
               color=['sample'],
               use_raw=False,
               frameon=False, 
               color_map='Spectral_r',
               size=30,
               add_outline=False,
               wspace = 0.5,
               ncols = 4,
#            save="_APRIL_28.pdf"
              )

sc.pl.umap(adata[adata.obs['sample']=='APRIL_28_IL18'],
               color=['sample'],
               use_raw=False,
               frameon=False, 
               color_map='Spectral_r',
               size=30,
               add_outline=False,
               wspace = 0.5,
               ncols = 4,
#            save="_APRIL_28_IL18.pdf"
              )

sc.pl.umap(adata[adata.obs['sample']=='APRIL_BB'],
               color=['sample'],
               use_raw=False,
               frameon=False, 
               color_map='Spectral_r',
               size=30,
               add_outline=False,
               wspace = 0.5,
               ncols = 4,
#            save="_APRIL_BB.pdf"
              )

sc.pl.umap(adata[adata.obs['sample']=='APRIL_BB_IL18'],
               color=['sample'],
               use_raw=False,
               frameon=False, 
               color_map='Spectral_r',
               size=30,
               add_outline=False,
               wspace = 0.5,
               ncols = 4,
#            save="_APRIL_BB_IL18.pdf"
              )

# create new anndata for each cell type

In [ ]:
CD8 = adata[adata.obs['leiden_0.1']=='0']
CD4 = adata[adata.obs['leiden_0.1']=='1']

In [ ]:
CD4

In [ ]:
CD8

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(CD4, n_comps=200, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
rng = np.random.RandomState(42)
sc.tl.pca(CD8, n_comps=200, svd_solver='arpack', random_state=rng, use_highly_variable=True)

In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(CD4, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    CD4, 
    color=['sample','leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8', 'leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r',
    use_raw=False,
    ncols=4,
    wspace = 0.3,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    sc.tl.leiden(CD8, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=False, transparent=True)
sc.pl.umap(
    CD8, 
    color=['sample','leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8', 'leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r',
    use_raw=False,
    ncols=5,
    wspace = 0.3,
    outline_width=[0.6, 0.05],
    size=15,
    frameon=False,
    add_outline=True,
    sort_order = False
)

In [ ]:
CD4_to_h5ad = '/Users/brandonng/Google Drive/Weill Cornell/LAB/folder containing all data from Mac pro/experiments and data/single cell/scotts single cell data/SJ_single_cell_CD4.h5ad'


In [ ]:
CD4.write(CD4_to_h5ad)

In [ ]:
# code to read back your saved h5ad object
CD4 = sc.read_h5ad(CD4_to_h5ad)
CD4.uns['log1p']["base"] = None

In [ ]:
CD8_to_h5ad = '/Users/brandonng/Google Drive/Weill Cornell/LAB/folder containing all data from Mac pro/experiments and data/single cell/scotts single cell data/SJ_single_cell_CD8.h5ad'


In [ ]:
CD8.write(CD8_to_h5ad)

In [ ]:
# code to read back your saved h5ad object
CD8 = sc.read_h5ad(CD8_to_h5ad)
CD8.uns['log1p']["base"] = None

In [ ]:
#create new anndata containing just 2 of 4 different groups
adata_APRIL_28_vs_28_IL18 = adata[(adata.obs['sample']=='APRIL_28') |
                                          (adata.obs['sample']=='APRIL_28_IL18')]
                                    

adata_APRIL_28_vs_BB = adata[(adata.obs['sample']=='APRIL_28') |
                                          (adata.obs['sample']=='APRIL_BB')]

adata_APRIL_28_vs_BB_IL18 = adata[(adata.obs['sample']=='APRIL_28') |
                                          (adata.obs['sample']=='APRIL_BB_IL18')]

adata_APRIL_28_IL18_vs_BB_IL18 = adata[(adata.obs['sample']=='APRIL_28_IL18') |
                                          (adata.obs['sample']=='APRIL_BB_IL18')]
                                    

adata_APRIL_BB_vs_BB_IL18 = adata[(adata.obs['sample']=='APRIL_BB') |
                                          (adata.obs['sample']=='APRIL_BB_IL18')]

In [ ]:
subset_to_h5ad = '/Users/brandonng/Google Drive/Weill Cornell/LAB/folder containing all data from Mac pro/experiments and data/single cell/20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/APRIL_28_IL18_vs_BB_IL18.h5ad'


In [ ]:
# adata_APRIL_BB_vs_BB_IL18.write(subset_to_h5ad)

In [ ]:
# code to read back your saved h5ad object
adata_APRIL_28_IL18_vs_BB_IL18 = sc.read_h5ad(subset_to_h5ad)
adata_APRIL_28_IL18_vs_BB_IL18.uns['log1p']["base"] = None

In [ ]:
adata_APRIL_28_IL18_vs_BB_IL18.obs

# running wilcoxon gene ranking for each cell type

In [ ]:
writer = pd.ExcelWriter('../20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/all_cells_wilcoxon.xlsx', engine='xlsxwriter')

sc.tl.rank_genes_groups(adata, 'sample', method='wilcoxon', use_raw=False)
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).to_excel(writer, sheet_name='Sheet1')
        
# writer.close()

In [ ]:
writer = pd.ExcelWriter('../20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/APRIL_BB_vs_BB_IL18_wilcoxon.xlsx', engine='xlsxwriter')

sc.tl.rank_genes_groups(adata_APRIL_BB_vs_BB_IL18, 'sample', method='wilcoxon', use_raw=False)
result = adata_APRIL_BB_vs_BB_IL18.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).to_excel(writer, sheet_name='Sheet1')
        
# writer.close()


In [ ]:
writer = pd.ExcelWriter('../20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/APRIL_28_IL18_vs_BB_IL18_wilcoxon.xlsx', engine='xlsxwriter')

sc.tl.rank_genes_groups(adata_APRIL_28_IL18_vs_BB_IL18, 'sample', method='wilcoxon', use_raw=False)
result = adata_APRIL_28_IL18_vs_BB_IL18.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).to_excel(writer, sheet_name='Sheet1')
        
# writer.close()


In [ ]:
writer = pd.ExcelWriter('../20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/APRIL_28_vs_BB_IL18_wilcoxon.xlsx', engine='xlsxwriter')

sc.tl.rank_genes_groups(adata_APRIL_28_vs_BB_IL18, 'sample', method='wilcoxon', use_raw=False)
result = adata_APRIL_28_vs_BB_IL18.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).to_excel(writer, sheet_name='Sheet1')
        
# writer.close()

In [ ]:
writer = pd.ExcelWriter('../20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/APRIL_28_vs_28_IL18_wilcoxon.xlsx', engine='xlsxwriter')

sc.tl.rank_genes_groups(adata_APRIL_28_vs_28_IL18, 'sample', method='wilcoxon', use_raw=False)
result = adata_APRIL_28_vs_28_IL18.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).to_excel(writer, sheet_name='Sheet1')
        
# writer.close()

In [ ]:
writer = pd.ExcelWriter('../20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/APRIL_28_vs_BB.xlsx', engine='xlsxwriter')

sc.tl.rank_genes_groups(adata_APRIL_28_vs_BB, 'sample', method='wilcoxon', use_raw=False)
result = adata_APRIL_28_vs_BB.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).to_excel(writer, sheet_name='Sheet1')
        
# writer.close()

In [ ]:
writer = pd.ExcelWriter('../20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/APRIL_28_18_vs_BB_18.xlsx', engine='xlsxwriter')

sc.tl.rank_genes_groups(adata_APRIL_28_IL18_vs_BB_IL18, 'sample', method='wilcoxon', use_raw=False)
result = adata_APRIL_28_IL18_vs_BB_IL18.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).to_excel(writer, sheet_name='Sheet1')
        
# writer.close()

## GSEApy

In [ ]:
import gseapy
from gseapy import gseaplot

In [ ]:
df = pd.read_csv('../20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/GSEA/inputs/APRIL_BB_IL18_vs_rest_wilcoxon_rnk.rnk',
                 sep='\t',
                   header=0)


In [ ]:
df.head(200)

In [ ]:
GSEA_HALLMARK_APRIL_BB_IL18_vs_rest = gseapy.prerank(df, seed = 42,
                                gene_sets="../20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/GSEA/mh.all.v2023.2.Mm.symbols.gmt")


In [ ]:
GSEA_HALLMARK_APRIL_BB_IL18_vs_rest

In [ ]:
GSEA_HALLMARK_APRIL_BB_IL18_vs_rest_1 = GSEA_HALLMARK_APRIL_BB_IL18_vs_rest

In [ ]:
GSEA_HALLMARK_APRIL_BB_IL18_vs_rest.res2d.head(100)

In [ ]:
GSEA_HALLMARK_APRIL_BB_IL18_vs_rest_1_sorted = GSEA_HALLMARK_APRIL_BB_IL18_vs_rest_1.res2d.sort_values('FDR q-val', ascending = True).reset_index(drop = True)


In [ ]:
GSEA_HALLMARK_APRIL_BB_IL18_vs_rest_1_sorted.head(200)

In [ ]:
terms = GSEA_HALLMARK_APRIL_BB_IL18_vs_rest.res2d.Term

axs = GSEA_HALLMARK_APRIL_BB_IL18_vs_rest.plot(terms=terms[2]) # v1.0.5

In [ ]:
#GSEA aggregate dotplots

In [ ]:
%%R -w 15 -h 25 -u cm

fromNetwork <- read_excel("/Users/brandonng/Google Drive/Weill Cornell/LAB/folder containing all data from Mac pro/experiments and data/single cell/20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/GSEA/cytoscape/M5_GOBP_comparisons_updated_FDRq_filt_reduced.xlsx", sheet="Sheet1")
fromNetwork_clean <- select(fromNetwork, -contains(c("::Dataset_Chart", "::Genes", "::GS_DESCR", "::Name", "shared name", "selected", 
                                                     "::GS_Type", "Colouring", "::ES", "::fwer_qvalue", "::pvalue")))
#fromNetwork_clean = fromNetwork_clean  %>% group_by(Group) %>% slice_min(order_by = minFDR, n = 5)

forDotplot <- as_tibble(lapply(fromNetwork_clean[,c("EnrichmentMap::gs_size", "name" , "grouping factor", "min_FDR")], rep, ncol(select(fromNetwork_clean,starts_with("EnrichmentMap::fdr_qvalue")))))


tmpFDR = NULL
FDR=NULL
tmpNES = NULL
NES=NULL
tmp_subset = NULL
subset=NULL

# append (in order) all FDR columns and all NES columns
for (i in c(1:ncol(select(fromNetwork_clean ,starts_with("EnrichmentMap::fdr_qvalue"))))) {
  tmpFDR = select(fromNetwork_clean,starts_with("EnrichmentMap::fdr_qvalue"))[i]
  colnames(tmpFDR) = "FDR"
  FDR = rbind(FDR, tmpFDR)
}

for (i in c(1:ncol(select(fromNetwork_clean,starts_with("EnrichmentMap::NES"))))) {
  tmpNES = select(fromNetwork_clean,starts_with("EnrichmentMap::NES"))[i]
  colnames(tmpNES) = "NES"
  NES = rbind(NES, tmpNES) 
}

forDotplot$NES = NES$NES
forDotplot$FDR = FDR$FDR

forDotplot$subset = rep(str_replace(colnames(select(fromNetwork_clean,starts_with("EnrichmentMap::NES"))), "EnrichmentMap::NES ", ""), each=nrow(fromNetwork_clean))
colnames(forDotplot) <- c("gs_size", "name", "group", "minFDR", "NES", "FDR", "subset")
forDotplot$FDR[forDotplot$FDR>0.05] <- NA
forDotplot$FDRtr = -log(forDotplot$FDR+0.00001,10)

print(ggdotchart(forDotplot,  x='name', y='subset', group = 'group',  size='FDRtr', rotate=TRUE, color = 'NES', xlab = "", ylab = "", sorting = "none") +
  scale_colour_gradientn(colours = dichromat::colorschemes$DarkRedtoBlue.18) +
  theme_pubclean() + theme(axis.text.x = element_text(face ="bold", angle = 90), axis.text.y = element_text(size =6)) +
  theme(legend.position = "right", panel.background = element_rect(colour = "black",size = 1, linetype = "solid"))) #+ 

# ggsave("M5_GOBP_comparisons_updated_FDRq_filt_reduced.pdf")


# heatmaps

In [ ]:
T_cell_genes = ['Il18', 'Pdcd1', 'Havcr2', 'Lag3', 'Entpd1', 'Tigit', 'Cd244a', 'Cd160', 'Tox', 'Eomes', 'Tbx21', 'Nfatc1',
                'Zeb2', 'Bcl2', 'Batf', 'Il21', 'Il21r', 'Cd47', 'Ltb',  'H2-Ab1', 'H2-K1', 'H2-D1',
                'B2m', 'Stat5a', 'Il2', 'Nfkb2', 'Relb', 'Junb', 'Ly6c1', 'Ly6a', 'Cd69',
                 'Id2', 'Actg1']

combined_phenotypes = ['Il18', 'Pdcd1', 'Havcr2', 'Lag3', 'Entpd1', 'Tigit', 'Cd244a', 'Cd160', 'Tnfsf10',
                       'Tox', 'Nr4a1', 'Nr4a2', 'Eomes', 'Tbx21', 'Nfatc1', 'Zeb2', 'Cd200', 
                       'Bad', 'Bid', 'Morrbid', 'Siva1', 'Bax',
                       'Scd2', 'Mif', 'Mgll', 'Gpx4', 'Cd74',
                       'Gapdh', 'Aldoa', 'Aldoc', 'Pgk1', 'Pgam1', 'Pkm', 'Gpi1', 'Slc2a3',
                       'Bcl2', 'Stat5a', 'Il2', 'Bach2', 'Il21', 'Il21r', 'Nfkb2', 'Nfkbia', 'Relb', 'Junb', 'Ltb', 
                       'Isg15', 'H2-Ab1', 'H2-K1', 'H2-D1', 'B2m', 'Batf', 'Ly6c1', 'Ly6a', 'Cd69', 
                  'Actg1', 'Itgb2', 'Itgb3'
                      ]

In [ ]:
sc.set_figure_params(scanpy=True, fontsize=10, dpi_save=300, transparent=True)
all_cells_heatmap = sc.pl.matrixplot(adata, combined_phenotypes,
                                     groupby='replicate',
                                     figsize=(6, 9),
                                     cmap='coolwarm', 
                                    standard_scale='var',
                                     use_raw=False,
                                     dendrogram=False,
                                      swap_axes=True,
#                                        save="_heatmap_CI82_combined_phenotypes.pdf"
                                       )

# gene scores
### using scanpy.tl.score_genes

In [ ]:
T_cell_exh_genes = ['Pdcd1', 'Havcr2', 'Lag3', 'Entpd1', 'Cd244a', 'Cd160', 'Tox', 'Eomes', 'Tbx21', 'Nfatc1',
                'Zeb2', 'Nr4a1', 'Nr4a2']

# T_cell_persistence_genes = 

In [ ]:
sc.tl.score_genes(CD8, gene_list = T_cell_exh_genes,  score_name='T_cell_exh_score', use_raw=False)


In [ ]:
sc.pl.violin(CD8, 'T_cell_exh_score' ,groupby='sample', rotation=90, jitter=False, )

In [ ]:
sc.pl.umap(adata,
           color=['sample', 'leiden_1.0', 'T_cell_exh_score'], cmap='Reds',
           vmax=0.65, vmin=0, wspace=0.4,
           size = 40, frameon=False)

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, vector_friendly=True, transparent=True)

sc.pl.umap(adata[adata.obs['sample']=='APRIL_28'],
           color='T_cell_exh_score', cmap='viridis',
          vmax=0.6, vmin=-0.3,
           size = 50, frameon=False,
#            save="_APRIL_28_exh_score.pdf"
          )

sc.pl.umap(adata[adata.obs['sample']=='APRIL_28_IL18'],
           color='T_cell_exh_score', cmap='viridis',
         vmax=0.6, vmin=-0.3,
           size = 50, frameon=False,
#            save="_APRIL_28_IL18_exh_score.pdf"
          )

sc.pl.umap(adata[adata.obs['sample']=='APRIL_BB'],
           color='T_cell_exh_score', cmap='viridis',
           vmax=0.6, vmin=-0.3,
           size = 50, frameon=False,
#            save="_APRIL_BB_exh_score.pdf"
          )

sc.pl.umap(adata[adata.obs['sample']=='APRIL_BB_IL18'],
           color='T_cell_exh_score', cmap='viridis',
           vmax=0.6, vmin=-0.3,
           size = 50, frameon=False,
#            save="_APRIL_BB_IL18_exh_score.pdf"
          )


In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, vector_friendly=True, transparent=True)

sc.pl.umap(adata,
           color='T_cell_exh_score', cmap='Reds',
          vmax=0.65, vmin=0,
           size = 50, frameon=False,
           save="_all_cells_exh_score.pdf"
          )


In [ ]:
CD8_Exh_score = CD8.obs[['sample', 'T_cell_exh_score']].copy()


In [ ]:
CD8_Exh_score

In [ ]:
CD8_Exh_score.to_excel("/Users/brandonng/Google Drive/Weill Cornell/LAB/folder containing all data from Mac pro/experiments and data/single cell/20231211_CI82_APRIL_28_IL18_BB_IL18_scRNA_rebuttal/CD8_Exh_score.xlsx", sheet_name='Sheet1')


<a id='magic'></a>

## System log

Date, time stamps, version numbers, and hardware information for version control.

In [ ]:
%load_ext watermark

In [ ]:
%watermark